In the previous Notebooks i've established that using a Linear regression to predict Rank is my best bet.

This method is called a pointwise ranking method. and the "learn to rank" Theory proposes more ranking methods that i'll try to explore in this norebook. 

    -Pairwise ranking 
    -Listwise ranking

Let's start with Pairwise ranking there are various methods all from Microsoft researcher Burges
RankNet, LambdaRank, LambdaRankMart

In [1]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')
Y_train['Rank']= Y_train['TARGET'].rank()
Y_train

,ID,TARGET,Rank
0,1054,0.028313,807.0
1,2049,-0.112516,532.0
2,1924,-0.180840,427.0
3,297,-0.260356,343.0
4,1101,-0.071733,593.0
...,...,...,...
1489,459,-0.172597,440.0
1490,1674,-0.063546,609.0
1491,748,0.151797,1016.0
1492,1454,-0.640917,163.0


In [4]:
df=create_pairwise_dataset(Y_train)
print(df)


            ID1     ID2  Comparison
0        1054.0  2049.0         2.0
1        1054.0  1924.0         2.0
2        1054.0   297.0         2.0
3        1054.0  1101.0         2.0
4        1054.0  1520.0         0.0
...         ...     ...         ...
1115266  1674.0  1454.0         2.0
1115267  1674.0   359.0         2.0
1115268   748.0  1454.0         2.0
1115269   748.0   359.0         2.0
1115270  1454.0   359.0         0.0

[1115271 rows x 3 columns]


In [22]:
import numpy.random as np_random
Comp_map = {1.0:np_random.uniform(0.66,1), 0.0: np_random.uniform(0,0.33), 0.5:np_random.uniform(0.33,0.66) } 
df['Comparison']=df['Comparison'].map(Comp_map)
df

,ID1,ID2,Comparison
0,1054.0,2049.0,0.970357
1,1054.0,1924.0,0.970357
2,1054.0,297.0,0.970357
3,1054.0,1101.0,0.970357
4,1054.0,1520.0,0.191529
...,...,...,...
1115266,1674.0,1454.0,0.970357
1115267,1674.0,359.0,0.970357
1115268,748.0,1454.0,0.970357
1115269,748.0,359.0,0.970357


In [23]:
remade=reconstruct_original_dataset(df)
remade

,ID,Rank_recreated
0,0.0,455
1,2.0,778
2,3.0,364
3,4.0,810
4,5.0,589
...,...,...
1489,2142.0,1357
1490,2143.0,1314
1491,2144.0,571
1492,2145.0,434


In [24]:
sorted=remade.sort_values(by=['ID'], key=lambda x: x.map({k: i for i, k in enumerate(Y_train['ID'].tolist())}))
sorted['ID'].tolist()==Y_train['ID'].tolist()

True

In [25]:
from scipy.stats import spearmanr
print("TARGET AND reconstructed Rank :" ,spearmanr(sorted["Rank_recreated"], Y_train["TARGET"]).correlation)


TARGET AND reconstructed Rank : 0.9999998515590814


Predicting Pairwise ranking accurately -> predicting Ranking accurately -> high spearman score

meaning our method of creating pairwise ranking and prediction a pairwise ranking ID by ID might help determine a more accurate ranking. Now let's Get a  X_train_pairwise that matches our Y_train_Pairwise

In [6]:
X_train_pairwise=get_training_pairwise(X_train)

Country remapped


In [13]:
print(sorted(Y_train['TARGET'].tolist(),reverse=True)[:10])

[7.786577856046542, 7.138604025388376, 6.836019876868637, 6.681753850368199, 6.151133366269985, 6.094297093901249, 6.009904429692065, 5.57011518732641, 5.0164912760257, 5.015579833097478]


In [1]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')

from sklearn.linear_model import LinearRegression
from utils.compute_model_performance import *

model=Model(LinearRegression,LinearRegression)
model.get_data_and_order(X_train,Y_train) 
# model.Compute(iterations=100,custom_group=2,group=True)
model.Compute(iterations=1,columns='Rank_recreated',split=False)



X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done
list length is 69751
error_val -0.04701839328939873
accuracy_val [0.40577196025863427, (0.4740220488759679, 34741), (0.3380466763790099, 35007)]
list length is 626640
error_train -4.811549331934183e-16
accuracy_train [0.44191561343035873, (0.4974421885542538, 326060), (0.38169876796236346, 300559)]
        ID1     ID2  Comparison     ID1     ID2  Comparison
0     256.0   276.0    0.608195   256.0   276.0         2.0
1     256.0  1150.0    1.086770   256.0  1150.0         0.0
2     256.0   347.0    0.917707   256.0   347.0         2.0
3     256.0  1109.0    0.985450   256.0  1109.0         2.0
4     256.0  1898.0    0.694000   256.0  1898.0         2.0
..      ...     ...         ...     ...     ...         ...
995  1150.0   343.0    0.756570  1150.0   343.0         2.0
996  1150.0  1249.0    0.596868  1150.0  1249.0         2.0
997  1150.0   737.0    0.610526  11

In [2]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')

from sklearn.linear_model import LogisticRegression
from utils.compute_model_performance import *

model=Model(LogisticRegression,LogisticRegression,max_iter=1000000,Solver='lbfgs')
model.get_data_and_order(X_train,Y_train) 
# model.Compute(iterations=100,custom_group=2,group=True)
model.Compute(iterations=1,columns='Rank_recreated',split=False)



TypeError: Model.__init__() got an unexpected keyword argument 'solver'

In [5]:
from mord import OrdinalRidge
from utils.compute_model_performance import *
model=Model(OrdinalRidge,OrdinalRidge)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=2,columns='Rank_recreated',split=False)

X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done


/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but OrdinalRidge was fitted without feature names
  warnings.warn(
/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but OrdinalRidge was fitted without feature names
  warnings.warn(


list length is 69751
error_val (-0.031211022064199796, [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0])
accuracy_val [0.03346188585109891, 0.04778986345753298, 0.01930073905628198, 1.0]
list length is 626640
error_train (0.0028708668453976765, [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.

/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but OrdinalRidge was fitted without feature names
  warnings.warn(
/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but OrdinalRidge was fitted without feature names
  warnings.warn(


list length is 69751
error_val (-0.04789895485369385, [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0])
accuracy_val [0.03852274519361729, 0.04162912236438998, 0.03565942506034672, 1.0]
list length is 626640
error_train (-0.0047060513213328225, [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.

In [7]:
from sklearn.ensemble import RandomForestRegressor
model=Model(RandomForestRegressor,RandomForestRegressor,depth=13,n_estimators=2)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=1,columns='Rank_recreated',split=False)

X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done
list length is 69751
error_val (-0.043252548783399083, [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.9971509971509973, 1.9971509971509973, 1.9940968122786304, 1.9940968122786304, 1.9922813036020584, 1.9922813036020584, 1.9922813036020584, 1.9922813036020584, 1.9902912621359223, 1.989648033126294, 1.989648033126294, 1.989648033126294, 1.989648033126294, 1.989648033126294, 1.989648033126294, 1.989648033126294, 1.989648033126294, 1.989648033126294, 1.989648033126294, 1.9876543209876543, 1.9876543209876543])
accuracy_val 

In [10]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from utils.compute_model_performance import *

# model=Model(RandomForestRegressor,RandomForestRegressor,depth=3,n_estimators=10)
# model.get_data_and_order(X_train,Y_train) 
# model.Compute(iterations=1,columns='Rank_recreated',split=False)

model=Model(RandomForestClassifier,RandomForestClassifier,depth=10,n_estimators=5)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=1,columns='Rank_recreated',split=False)

X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done
list length is 69751
error_val (-0.04143309773336583, [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0])
accuracy_val [0.530200283866898, 0.5290898704110165, 0.0]
list length is 626640
error_train (0.0012894165709179113, [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.

In [2]:
import xgboost as xgb

model=Model(xgb.XGBClassifier,xgb.XGBClassifier,depth=10,n_estimators=50)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=1,columns='Rank_recreated',split=False)

X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done


TypeError: 'NoneType' object is not subscriptable

Conclusion Pairwise prediction with "traditional" means is not accurate enough to construct a robust ranking representation


Actually let's study our required accuracy to get an acceptable validation

In [6]:
sub_df=select_precentage_of_df(df,95)
sub_df['Comparison']= np.random.permutation(sub_df['Comparison'].values)
df2 = pd.merge(df, sub_df, on=['ID1','ID2'], how='left', suffixes=('', '_modified'))
# Replace the original 'Value' column with the modified values
df2['Comparison'] = df2['Comparison_modified'].fillna(df2['Comparison']).astype(int)
df2.drop(['Comparison_modified'], axis=1, inplace=True)
print(df2)
print("accuract between df and df scrambled is ",accuracy(df['Comparison'],df2['Comparison']))
remade=reconstruct_original_dataset(df2)
sorted=remade.sort_values(by=['ID'], key=lambda x: x.map({k: i for i, k in enumerate(Y_train['ID'].tolist())}))
# print(np.unique(sorted['Rank_recreated']))
print(pd.concat((sorted.reset_index(drop=True),Y_train.reset_index(drop=True)),axis=1))
from scipy.stats import spearmanr
print("TARGET AND reconstructed Rank :" ,spearmanr(sorted["Rank_recreated"], Y_train["TARGET"]).correlation)



            ID1     ID2  Comparison
0        1054.0  2049.0           2
1        1054.0  1924.0           2
2        1054.0   297.0           2
3        1054.0  1101.0           2
4        1054.0  1520.0           0
...         ...     ...         ...
1115266  1674.0  1454.0           2
1115267  1674.0   359.0           0
1115268   748.0  1454.0           0
1115269   748.0   359.0           2
1115270  1454.0   359.0           2

[1115271 rows x 3 columns]
accuract between df and df scrambled is  [0.524919055547934, (0.5288227059850982, 562213), (0.5209895608240674, 553013), (0.044444444444444446, 45)]
          ID  Rank_recreated    ID    TARGET
0     1054.0             753  1054  0.028313
1     2049.0             752  2049 -0.112516
2     1924.0             726  1924 -0.180840
3      297.0             704   297 -0.260356
4     1101.0             717  1101 -0.071733
...      ...             ...   ...       ...
1489   459.0             762   459 -0.172597
1490  1674.0             789  1